In [1]:
# load packages
# load libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
# load train and test users data
train_users = pd.read_csv("train_users.csv")
test_users = pd.read_csv("test_users.csv")

# load train and test items data
train_items = pd.read_csv("train_items.csv")
test_items = pd.read_csv("test_items.csv")

# load interactions
train_interactions = pd.read_csv("train_interactions.csv")

In [3]:
from scipy.sparse import csr_matrix
from lightfm import LightFM
from lightfm.data import Dataset

x# Initialize the LightFM Dataset object
dataset = Dataset()

# Fit the dataset with user IDs and item IDs from the interaction data
dataset.fit(
    users=train_interactions['user_id'].unique(),
    items=train_items['item_ID'].unique(),
    item_features=train_items.columns.drop(['item_ID', 'item_name']).tolist()
)

# Build interaction matrix
(interactions, _) = dataset.build_interactions(
    [(row['user_id'], row['item_ID']) for _, row in train_interactions.iterrows()]
)

# Build item features matrix
item_features = dataset.build_item_features(
    [(row['item_ID'], row.drop(['item_ID', 'item_name']).values.tolist())
     for _, row in train_items.iterrows()]
)

# Initialize the LightFM model
model = LightFM(loss='warp')  # WARP loss for ranking optimization

# Train the model
model.fit(interactions, item_features=item_features, epochs=10, num_threads=4)

# Model is trained; now generating recommendations.
# Predict scores for a sample user (e.g., user_id=209227)
sample_user_id = 209227
item_ids = train_items['item_ID'].unique()

# Predict scores
scores = model.predict(
    user_ids=sample_user_id,
    item_ids=item_ids,
    item_features=item_features
)

# Rank items by score and display top 5 recommendations
recommended_items = sorted(zip(item_ids, scores), key=lambda x: -x[1])[:5]
recommended_items


/Users/junjiezhang/opt/anaconda3/lib/python3.9/site-packages/lightfm/_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


ValueError: Feature FENDI not in feature mapping. Call fit first.

In [9]:
# drop columns that are not needed
train_items = train_items[['item_ID', 'item_name', 'brand', 'channel', 'unit_price', 'category',
        'gender', 'class_map', 'tag', 'price_range', 'country',
       'style_50']].copy()
       

item_ID               0.000000
item_name             0.000000
brand                 0.000000
channel               0.000000
unit_price            0.000000
category              0.000000
size                  0.001562
color                 0.002032
discount              0.000030
gender                0.000000
class_map             0.000000
bk_sku_id             0.483507
category.1            0.483507
style                 0.488533
brand_location_map    0.387735
materials_map         0.484178
sex                   0.484839
country_size          0.462385
country_in_stock      0.522469
tag                   0.000000
price_range           0.342567
country               0.361737
style_50              0.000000
color_50              0.002332
dtype: float64

In [17]:
train_items = train_items[['item_ID', 'item_name', 'brand', 'channel', 'unit_price', 'category',
        'gender', 'class_map', 'tag', 'price_range', 'country',
       'style_50']].copy()

In [18]:
train_items.head()

,item_ID,item_name,brand,channel,unit_price,category,gender,class_map,tag,price_range,country,style_50
0,1001920,Fendi女子皮制鞋12184-X3474,FENDI,17,299.0,单鞋,女,鞋靴,奢华&经典&现代,5.0,意大利 (IT),奢华
1,1003191,【自营】BCBG-MAXAZRIA女士皮制手提书包 【特卖货品，缺货退款，延迟发货见谅】,BCBG,15,89.0,手提包,女,箱包,高端&时尚&女性化,NaN,NaN,优雅
2,1003194,BCBG-MAXAZRIA女士皮制肩包 【特卖货品，缺货退款，延迟发货见谅】,BCBG,15,19.0,单肩包,女,箱包,高端&时尚&女性化,NaN,NaN,优雅
3,1003197,【自营】BCBG-MAXAZRIA女士皮制邮差包,BCBG,85,99.0,多种背包方式,女,帽子,高端&时尚&女性化,NaN,NaN,优雅
4,1003210,BCBG-MAXAZRIA女士聚酯材质DEMI包,BCBG,15,29.0,多种背包方式,女,帽子,高端&时尚&女性化,NaN,NaN,优雅


In [20]:
# Import necessary libraries
import pandas as pd
from lightfm import LightFM
from lightfm.data import Dataset

# Step 1: Load the datasets
# Load interaction data
train_interactions = pd.read_csv("train_interactions.csv")

# Load item data
train_items = pd.read_csv("train_items.csv")

# Optional: If you have a user profile table
#user_profile = pd.read_csv("user_profile.csv")

# Step 2: Initialize the LightFM Dataset object
dataset = Dataset()

# Step 3: Fit the Dataset with unique user IDs and item IDs
# This defines the universe of users and items that the model will work with
dataset.fit(
    users=train_users['user_id'].unique(),
    items=train_items['item_ID'].unique(),
    # Optional: Define the available user and item features
  #  user_features=None,  # Replace with user_profile.columns.drop('user_id') if available
    item_features=train_items.columns.drop(['item_ID', 'item_name']).tolist()
)

# Step 4: Build the interaction matrix
# Use the user_id and item_ID from the interaction table to create the matrix
(interactions, _) = dataset.build_interactions(
    [(row['user_id'], row['item_ID']) for _, row in train_interactions.iterrows()]
)

# Step 5: Build the item features matrix
# Extract item features from the train_items table
item_features = dataset.build_item_features(
    [(row['item_ID'], row.drop(['item_ID', 'item_name']).values.tolist())
     for _, row in train_items.iterrows()]
)

# Optional: Build the user features matrix if you have user features
# user_features = dataset.build_user_features(
#     [(row['user_id'], row.drop('user_id').values.tolist()) for _, row in user_profile.iterrows()]
# )

# Step 6: Initialize and train the LightFM model
# Choose a loss function (e.g., 'warp' for ranking optimization)
model = LightFM(loss='warp')

# Fit the model
model.fit(
    interactions=interactions,           # Interaction matrix
    item_features=item_features,         # Item features matrix
    # user_features=user_features,       # Uncomment if user features are available
    epochs=10,                           # Number of training epochs
    num_threads=4                        # Number of threads for parallel training
)



ValueError: Feature FENDI not in feature mapping. Call fit first.

In [ ]:
# Step 7: Generate recommendations
# Predict scores for a sample user (e.g., user_id = 209227)
sample_user_id = 1
item_ids = train_items['item_ID'].unique()  # All unique item IDs

# Predict scores for all items for the sample user
scores = model.predict(
    user_ids=sample_user_id,
    item_ids=item_ids,
    item_features=item_features
    # user_features=user_features,       # Uncomment if user features are available
)

# Rank items by their predicted scores
recommended_items = sorted(zip(item_ids, scores), key=lambda x: -x[1])[:5]



In [ ]:
# Print the top 5 recommendations
print(f"Top recommendations for user {sample_user_id}:")
for item, score in recommended_items:
    print(f"Item ID: {item}, Predicted Score: {score}")

# Step 8: Evaluate the model
# Evaluate using precision@k
from lightfm.evaluation import precision_at_k

precision = precision_at_k(
    model,
    interactions=interactions,
    item_features=item_features,
    # user_features=user_features,       # Uncomment if user features are available
    k=5
).mean()

print(f"Precision@5: {precision}")

# Optional: Evaluate using recall@k
from lightfm.evaluation import recall_at_k

recall = recall_at_k(
    model,
    interactions=interactions,
    item_features=item_features,
    # user_features=user_features,       # Uncomment if user features are available
    k=5
).mean()

print(f"Recall@5: {recall}")
